### Importing Libraries to be used for the following code

In [25]:
import requests
import csv
import pandas as pd
from datetime import datetime

### Requesting data from the json link and importing data from excel

In [2]:
json_link = 'https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json'

response = requests.get(json_link)

if response.status_code == 200:
    json_data = response.json()
else:
    print(f'Failed to fetch data from JSON link. Status code: {response.status_code}')

In [11]:
country_df = pd.read_excel("Country-Code.xlsx")

### Defining a general recursive function that extracts information from a restaurant dictionary

The function takes in the dictionary and goes through it recursively until either there are no more nested list or dictionaries and or the keys to be extracted cannot be found any more

In [59]:
def extract_values(dictionary, keys_to_extract):
    result = {}

    def recursive_extract(data, keys):
        for key, value in data.items():
            if isinstance(value, dict):
                recursive_extract(value, keys)
            elif key in keys:
                result[key] = value

    recursive_extract(dictionary, keys_to_extract)
    return result

### Defining the function for question 1

The function directly writes to the csv while at the same time comparing the country_id to the data in the country excel sheet to get the country name for each restaurant entry

In [ ]:
def question_1(res_columns, filename, extraction_func=extract_values):
    with open(filename, mode='w', newline='') as res_file:
        writer = csv.DictWriter(res_file, fieldnames=res_columns)
        writer.writeheader()

        for dictionary in json_data:
            for restaurant_data in dictionary["restaurants"]:
                extracted_values = extraction_func(restaurant_data, res_columns)     

                country_info = country_df[country_df['Country Code'] == extracted_values['country_id']]
                if not country_info.empty:
                    extracted_values.update({
                        'country': country_info['Country'].values[0]
                    })
                writer.writerow(extracted_values)
                
    return f'The file is saved in {filename}'

res_columns_1 = ['res_id', 'name', 'country_id','country','city','votes','aggregate_rating','cuisines']

question_1(res_columns_1, 'question1.csv')

### Defining the function for question 2

This function is similar to that for question 1 but there is an added condition where only events that are held in **April 2019** are considered. Blank entries are given a NA value.

In [31]:
def question_2(res_columns, filename, extraction_func=extract_values):
    with open(filename, mode='w', newline='') as res_file:
        writer = csv.DictWriter(res_file, fieldnames=res_columns)
        writer.writeheader()

        for dictionary in json_data:
            for restaurant_data in dictionary["restaurants"]:
                extracted_values = extraction_func(restaurant_data, res_columns)     

                if 'start_date' in extracted_values:
                    start_date = datetime.strptime(extracted_values['start_date'], '%Y-%m-%d')

                    if start_date.year == 2019 and start_date.month == 4:

                        for i in res_columns: 
                            if i not in extracted_values:
                                extracted_values[i] = pd.NA

                        writer.writerow(extracted_values)
    return f'The file is saved in {filename}'

res_columns_2 = ['event_id','res_id','name','photos_url','title','start_date','end_date']

question_2(res_columns_2, 'question2.csv')

## Defining function for question 3

The function extracts the values similar to question 1 and 2. It then checks whether it is part of the rating text that we are looking out for and gets the range of the threshold for the different rating text

In [61]:
def question_3(res_columns, ratings_list, extraction_func=extract_values):
    temp = []
    for dictionary in json_data:
        for restaurant_data in dictionary["restaurants"]:
            extracted_values = extract_values(restaurant_data, res_columns)  
            
            if extracted_values['rating_text'] in ratings_list:
                temp.append(extracted_values)
    test_df = pd.DataFrame(temp)
    threshold = test_df.groupby('rating_text')['aggregate_rating'].agg(['min','max'])
    return threshold

res_columns_3 = ['aggregate_rating','rating_text']
ratings_list = ['Excellent','Very Good','Good','Average','Poor']

question_3(res_columns_3, ratings_list)

,min,max
rating_text,,
Average,2.5,3.4
Excellent,4.5,4.9
Good,3.5,3.9
Poor,2.2,2.2
Very Good,4.0,4.4
